# 🔧 环境配置和检查

#### 概述

本教程需要特定的环境配置以确保最佳学习体验。以下配置将帮助你：

- 使用统一的conda环境：激活统一的学习环境
- 通过国内镜像源快速安装依赖：配置pip使用清华镜像源
- 加速模型下载：设置HuggingFace镜像代理
- 检查系统配置：检查硬件和软件配置

#### 配置

- **所需环境及其依赖已经部署好**
- 在`Notebook`右上角选择`jupyter内核`为`python(agent101)`，即可执行


In [1]:
%%script bash

# 1. 激活 conda 环境 (仅对当前单元格有效)
eval "$(conda shell.bash hook)"
conda activate agent101

echo "========================================="
echo "== Conda 环境检查报告 (仅针对当前 Bash 子进程) =="
echo "========================================="

# 2. 检查当前激活的环境
CURRENT_ENV_NAME=$(basename $CONDA_PREFIX)

if [ "$CURRENT_ENV_NAME" = "agent101" ]; then
    echo "✅ 当前单元格已成功激活到 agent101 环境。"
    echo "✅ 正在使用的环境路径: $CONDA_PREFIX"
    echo ""
    echo "💡 提示: 后续的Python单元格将使用Notebook当前选择的Jupyter内核。"
    echo "   如果需要后续单元格也使用此环境，请执行以下操作:"
    echo "   1. 检查 Notebook 右上角是否已选择 'python(agent101)'。"
else
    echo "❌ 激活失败或环境名称不匹配。当前环境: $CURRENT_ENV_NAME"
    echo ""
    echo "⚠️ 严重提示: 建议将 Notebook 的 Jupyter **内核 (Kernel)** 切换为 'python(agent101)'。"
    echo "   (通常位于 Notebook 右上角或 '内核' 菜单中)"
    echo ""
    echo "📚 备用方法 (不推荐): 如果无法切换内核，则必须在**每个**代码单元格的头部重复以下命令:"
    echo ""
    echo "%%script bash"
    echo "# 必须在每个单元格都执行"
    echo "eval \"\$(conda shell.bash hook)\""
    echo "conda activate agent101"
fi

echo "========================================="


== Conda 环境检查报告 (仅针对当前 Bash 子进��) ==
✅ 当前单元格已成功激活到 agent101 环境。
✅ 正在使用的环境路径: /root/miniconda3/envs/agent101

💡 提示: 后续的Python单元格将使用Notebook当前选择的Jupyter��核。
   如果需要后续单元格也使用此环境，请执行以下操作:
   1. 检查 Notebook 右上角是否已选择 'python(agent101)'。


In [2]:
# 2. 设置pip 为清华源
%pip config list -v set global.index-url https://pypi.tuna.tsinghua.edu.cn/simple
%pip config list -v


Writing to /root/.config/pip/pip.conf
Note: you may need to restart the kernel to use updated packages.
ERROR: Need an action (debug, edit, get, list, set, unset) to perform.
Note: you may need to restart the kernel to use updated packages.


In [3]:
# 3. 设置HuggingFace代理
%env HF_ENDPOINT=https://hf-mirror.com
# 验证：使用shell命令检查
!echo $HF_ENDPOINT


env: HF_ENDPOINT=https://hf-mirror.com
https://hf-mirror.com


In [4]:
# 🔍 环境信息检查脚本
#
# 本脚本的作用：
# 1. 安装 pandas 库用于数据表格展示
# 2. 检查系统的各项配置信息
# 3. 生成详细的环境报告表格
#
# 对于初学者来说，这个步骤帮助你：
# - 了解当前运行环境的硬件配置
# - 确认是否满足模型运行的最低要求
# - 学习如何通过代码获取系统信息

# 安装 pandas 库 - 用于创建和展示数据表格
# pandas 是 Python 中最流行的数据处理和分析库
%pip install pandas==2.2.2 tabulate==0.9.0

import platform # 导入 platform 模块以获取系统信息
import os # 导入 os 模块以与操作系统交互
import subprocess # 导入 subprocess 模块以运行外部命令
import pandas as pd # 导入 pandas 模块，通常用于数据处理，这里用于创建表格
import shutil # 导入 shutil 模块以获取磁盘空间信息

# 获取 CPU 信息的函数，包括核心数量
def get_cpu_info():
    cpu_info = "" # 初始化 CPU 信息字符串
    physical_cores = "N/A"
    logical_cores = "N/A"

    if platform.system() == "Windows": # 如果是 Windows 系统
        cpu_info = platform.processor() # 使用 platform.processor() 获取 CPU 信息
        try:
            # 获取 Windows 上的核心数量 (需要 WMI)
            import wmi
            c = wmi.WMI()
            for proc in c.Win32_Processor():
                physical_cores = proc.NumberOfCores
                logical_cores = proc.NumberOfLogicalProcessors
        except:
            pass # 如果 WMI 不可用，忽略错误

    elif platform.system() == "Darwin": # 如果是 macOS 系统
        # 在 macOS 上使用 sysctl 命令获取 CPU 信息和核心数量
        os.environ['PATH'] = os.environ['PATH'] + os.pathsep + '/usr/sbin' # 更新 PATH 环境变量
        try:
            process_brand = subprocess.Popen(['sysctl', "machdep.cpu.brand_string"], stdout=subprocess.PIPE, stderr=subprocess.PIPE)
            stdout_brand, stderr_brand = process_brand.communicate()
            cpu_info = stdout_brand.decode().split(': ')[1].strip() if stdout_brand else "Could not retrieve CPU info"

            process_physical = subprocess.Popen(['sysctl', "hw.physicalcpu"], stdout=subprocess.PIPE, stderr=subprocess.PIPE)
            stdout_physical, stderr_physical = process_physical.communicate()
            physical_cores = stdout_physical.decode().split(': ')[1].strip() if stdout_physical else "N/A"

            process_logical = subprocess.Popen(['sysctl', "hw.logicalcpu"], stdout=subprocess.PIPE, stderr=subprocess.PIPE)
            stdout_logical, stderr_logical = process_logical.communicate()
            logical_cores = stdout_logical.decode().split(': ')[1].strip() if stdout_logical else "N/A"

        except:
            cpu_info = "Could not retrieve CPU info"
            physical_cores = "N/A"
            logical_cores = "N/A"

    else:  # Linux 系统
        try:
            # 在 Linux 上读取 /proc/cpuinfo 文件获取 CPU 信息和核心数量
            with open('/proc/cpuinfo') as f:
                physical_cores_count = 0
                logical_cores_count = 0
                cpu_info_lines = []
                for line in f:
                    if line.startswith('model name'): # 查找以 'model name'开头的行
                        if not cpu_info: # 只获取第一个 model name
                            cpu_info = line.split(': ')[1].strip()
                    elif line.startswith('cpu cores'): # 查找以 'cpu cores' 开头的行
                        physical_cores_count = int(line.split(': ')[1].strip())
                    elif line.startswith('processor'): # 查找以 'processor' 开头的行
                        logical_cores_count += 1
                physical_cores = str(physical_cores_count) if physical_cores_count > 0 else "N/A"
                logical_cores = str(logical_cores_count) if logical_cores_count > 0 else "N/A"
                if not cpu_info:
                     cpu_info = "Could not retrieve CPU info"

        except:
            cpu_info = "Could not retrieve CPU info"
            physical_cores = "N/A"
            logical_cores = "N/A"

    return f"{cpu_info} ({physical_cores} physical cores, {logical_cores} logical cores)" # 返回 CPU 信息和核心数量


# 获取内存信息的函数
def get_memory_info():
    mem_info = "" # 初始化内存信息字符串
    if platform.system() == "Windows":
        # 在 Windows 上不容易通过标准库获取，需要外部库或 PowerShell
        mem_info = "Requires external tools on Windows" # 设置提示信息
    elif platform.system() == "Darwin": # 如果是 macOS 系统
        # 在 macOS 上使用 sysctl 命令获取内存大小
        process = subprocess.Popen(['sysctl', "hw.memsize"], stdout=subprocess.PIPE, stderr=subprocess.PIPE) # 运行 sysctl 命令
        stdout, stderr = process.communicate() # 获取标准输出和标准错误
        mem_bytes = int(stdout.decode().split(': ')[1].strip()) # 解析输出，获取内存大小（字节）
        mem_gb = mem_bytes / (1024**3) # 转换为 GB
        mem_info = f"{mem_gb:.2f} GB" # 格式化输出
    else:  # Linux 系统
        try:
            # 在 Linux 上读取 /proc/meminfo 文件获取内存信息
            with open('/proc/meminfo') as f:
                total_mem_kb = 0
                available_mem_kb = 0
                for line in f:
                    if line.startswith('MemTotal'): # 查找以 'MemTotal' 开头的行
                        total_mem_kb = int(line.split(':')[1].strip().split()[0]) # 解析行，获取总内存（KB）
                    elif line.startswith('MemAvailable'): # 查找以 'MemAvailable' 开头的行
                         available_mem_kb = int(line.split(':')[1].strip().split()[0]) # 解析行，获取可用内存（KB）

                if total_mem_kb > 0:
                    total_mem_gb = total_mem_kb / (1024**2) # 转换为 GB
                    mem_info = f"{total_mem_gb:.2f} GB" # 格式化输出总内存
                    if available_mem_kb > 0:
                        available_mem_gb = available_mem_kb / (1024**2)
                        mem_info += f" (Available: {available_mem_gb:.2f} GB)" # 添加可用内存信息
                else:
                     mem_info = "Could not retrieve memory info" # 如果读取文件出错，设置错误信息

        except:
            mem_info = "Could not retrieve memory info" # 如果读取文件出错，设置错误信息
    return mem_info # 返回内存信息

# 获取 GPU 信息的函数，包括显存
def get_gpu_info():
    try:
        # 尝试使用 nvidia-smi 获取 NVIDIA GPU 信息和显存
        result = subprocess.run(['nvidia-smi', '--query-gpu=name,memory.total', '--format=csv,noheader'], capture_output=True, text=True)
        if result.returncode == 0: # 如果命令成功执行
            gpu_lines = result.stdout.strip().split('\n') # 解析输出，获取 GPU 名称和显存
            gpu_info_list = []
            for line in gpu_lines:
                name, memory = line.split(', ')
                gpu_info_list.append(f"{name} ({memory})") # 格式化 GPU 信息
            return ", ".join(gpu_info_list) if gpu_info_list else "NVIDIA GPU found, but info not listed" # 返回 GPU 信息或提示信息
        else:
             # 尝试使用 lshw 获取其他 GPU 信息 (需要安装 lshw)
            try:
                result_lshw = subprocess.run(['lshw', '-C', 'display'], capture_output=True, text=True)
                if result_lshw.returncode == 0: # 如果命令成功执行
                     # 简单解析输出中的 product 名称和显存
                    gpu_info_lines = []
                    current_gpu = {}
                    for line in result_lshw.stdout.splitlines():
                        if 'product:' in line:
                             if current_gpu:
                                 gpu_info_lines.append(f"{current_gpu.get('product', 'GPU')} ({current_gpu.get('memory', 'N/A')})")
                             current_gpu = {'product': line.split('product:')[1].strip()}
                        elif 'size:' in line and 'memory' in line:
                             current_gpu['memory'] = line.split('size:')[1].strip()

                    if current_gpu: # 添加最后一个 GPU 的信息
                        gpu_info_lines.append(f"{current_gpu.get('product', 'GPU')} ({current_gpu.get('memory', 'N/A')})")

                    return ", ".join(gpu_info_lines) if gpu_info_lines else "GPU found (via lshw), but info not parsed" # 如果找到 GPU 但信息无法解析，设置提示信息
                else:
                    return "No GPU found (checked nvidia-smi and lshw)" # 如果两个命令都找不到 GPU，设置提示信息
            except FileNotFoundError:
                 return "No GPU found (checked nvidia-smi, lshw not found)" # 如果找不到 lshw 命令，设置提示信息
    except FileNotFoundError:
        return "No GPU found (nvidia-smi not found)" # 如果找不到 nvidia-smi 命令，设置提示信息


# 获取 CUDA 版本的函数
def get_cuda_version():
    try:
        # 尝试使用 nvcc --version 获取 CUDA 版本
        result = subprocess.run(['nvcc', '--version'], capture_output=True, text=True)
        if result.returncode == 0: # 如果命令成功执行
            for line in result.stdout.splitlines():
                if 'release' in line: # 查找包含 'release' 的行
                    return line.split('release ')[1].split(',')[0] # 解析行，提取版本号
        return "CUDA not found or version not parsed" # 如果找不到 CUDA 或版本无法解析，设置提示信息
    except FileNotFoundError:
        return "CUDA not found" # 如果找不到 nvcc 命令，设置提示信息

# 获取 Python 版本的函数
def get_python_version():
    return platform.python_version() # 获取 Python 版本

# 获取 Conda 版本的函数
def get_conda_version():
    try:
        # 尝试使用 conda --version 获取 Conda 版本
        result = subprocess.run(['conda', '--version'], capture_output=True, text=True)
        if result.returncode == 0: # 如果命令成功执行
            return result.stdout.strip() # 返回 Conda 版本
        return "Conda not found or version not parsed" # 如果找不到 Conda 或版本无法解析，设置提示信息
    except FileNotFoundError:
        return "Conda not found" # 如果找不到 conda 命令，设置提示信息

# 获取物理磁盘空间信息的函数
def get_disk_space():
    try:
        total, used, free = shutil.disk_usage("/") # 获取根目录的磁盘使用情况
        total_gb = total / (1024**3) # 转换为 GB
        used_gb = used / (1024**3) # 转换为 GB
        free_gb = free / (1024**3) # 转换为 GB
        return f"Total: {total_gb:.2f} GB, Used: {used_gb:.2f} GB, Free: {free_gb:.2f} GB" # 格式化输出
    except Exception as e:
        return f"Could not retrieve disk info: {e}" # 如果获取信息出错，设置错误信息

# 获取环境信息
os_name = platform.system() # 获取操作系统名称
os_version = platform.release() # 获取操作系统版本
if os_name == "Linux":
    try:
        # 在 Linux 上尝试获取发行版和版本
        lsb_info = subprocess.run(['lsb_release', '-a'], capture_output=True, text=True)
        if lsb_info.returncode == 0: # 如果命令成功执行
            for line in lsb_info.stdout.splitlines():
                if 'Description:' in line: # 查找包含 'Description:' 的行
                    os_version = line.split('Description:')[1].strip() # 提取描述信息作为版本
                    break # 找到后退出循环
                elif 'Release:' in line: # 查找包含 'Release:' 的行
                     os_version = line.split('Release:')[1].strip() # 提取版本号
                     # 尝试获取 codename
                     try:
                         codename_info = subprocess.run(['lsb_release', '-c'], capture_output=True, text=True)
                         if codename_info.returncode == 0:
                             os_version += f" ({codename_info.stdout.split(':')[1].strip()})" # 将 codename 添加到版本信息中
                     except:
                         pass # 如果获取 codename 失败则忽略

    except FileNotFoundError:
        pass # lsb_release 可能未安装，忽略错误

full_os_info = f"{os_name} {os_version}" # 组合完整的操作系统信息
cpu_info = get_cpu_info() # 调用函数获取 CPU 信息和核心数量
memory_info = get_memory_info() # 调用函数获取内存信息
gpu_info = get_gpu_info() # 调用函数获取 GPU 信息和显存
cuda_version = get_cuda_version() # 调用函数获取 CUDA 版本
python_version = get_python_version() # 调用函数获取 Python 版本
conda_version = get_conda_version() # 调用函数获取 Conda 版本
disk_info = get_disk_space() # 调用函数获取物理磁盘空间信息


# 创建用于存储数据的字典
env_data = {
    "项目": [ # 项目名称列表
        "操作系统",
        "CPU 信息",
        "内存信息",
        "GPU 信息",
        "CUDA 信息",
        "Python 版本",
        "Conda 版本",
        "物理磁盘空间" # 添加物理磁盘空间
    ],
    "信息": [ # 对应的信息列表
        full_os_info,
        cpu_info,
        memory_info,
        gpu_info,
        cuda_version,
        python_version,
        conda_version,
        disk_info # 添加物理磁盘空间信息
    ]
}

# 创建一个 pandas DataFrame
df = pd.DataFrame(env_data)

# 打印表格
print("### 环境信息") # 打印标题
print(df.to_markdown(index=False)) # 将 DataFrame 转换为 Markdown 格式并打印，不包含索引


Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
Note: you may need to restart the kernel to use updated packages.
### 环境信息
| 项目         | 信息                                                                               |
|:-------------|:-----------------------------------------------------------------------------------|
| 操作系统     | Linux Ubuntu 22.04.4 LTS                                                           |
| CPU 信息     | 11th Gen Intel(R) Core(TM) i5-1135G7 @ 2.40GHz (1 physical cores, 2 logical cores) |
| 内存信息     | 3.78 GB (Available: 1.50 GB)                                                       |
| GPU 信息     | No GPU found (nvidia-smi not found)                                                |
| CUDA 信息    | CUDA not found                                                                     |
| Python 版本  | 3.10.18                                                                            |
| Conda 版本   | conda 24.4.0                                         

# 第六节：逐步思考


## 设置

运行以下设置单元格来加载你的API密钥并建立`get_completion`辅助函数。

In [5]:
# 安装OpenAI库
%pip install openai==1.107.0

# 🔧 OpenAI环境自动配置
# 此设置会自动从环境变量或IPython存储中加载配置


# 导入Python内置的正则表达式库
import re

# 🚀 使用统一配置管理系统
from config import setup_notebook_environment, print_config_info

# 自动设置OpenAI客户端和get_completion函数
# 优先级：环境变量 > IPython存储 > 默认值
try:
    client, get_completion = setup_notebook_environment()
    print("✅ 使用统一配置管理成功！")
except Exception as e:
    print(f"❌ 统一配置失败，回退到传统方式: {e}")
    
    # 回退到传统的配置方式
    import openai
    
    # 从IPython存储中检索API_KEY和MODEL_NAME变量
    %store -r API_KEY
    %store -r MODEL_NAME

    # 如果没有设置MODEL_NAME，使用默认值
    try:
        MODEL_NAME
    except NameError:
        MODEL_NAME = "大模型-3.5-turbo"  # 默认使用gpt-4o模型

    # 创建OpenAI客户端
    client = openai.OpenAI(api_key=API_KEY)

    def get_completion(prompt: str, system_prompt=""):
        """
        获取大模型的完成响应
        
        参数:
            prompt (str): 用户提示
            system_prompt (str): 系统提示（可选）
        
        返回:
            str: 大模型的响应文本
        """
        # 构建消息列表
        messages = []
        
        # 如果有系统提示，添加系统消息
        if system_prompt:
            messages.append({"role": "system", "content": system_prompt})
        
        # 添加用户消息
        messages.append({"role": "user", "content": prompt})
        
        # 调用OpenAI API
        response = client.chat.completions.create(
            model=MODEL_NAME,              # 模型名称 (gpt-4o 或 deepseek-r1)
            messages=messages,             # 消息列表
            max_completion_tokens=2000,    # 最大token数
            temperature=0.0               # 温度参数，0表示更确定性
        )
        return response.choices[0].message.content
    
    print("⚠️  使用传统配置方式，建议配置环境变量以获得更好体验")

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
Note: you may need to restart the kernel to use updated packages.
Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
Note: you may need to restart the kernel to use updated packages.
✅ OpenAI环境设置完成!
🔧 OpenAI API 配置信息:
  📡 配置来源: 环境变量 + 自定义API地址: https://api.apiyi.com/v1
  🤖 模型: deepseek-chat
  🌐 API地址: https://api.apiyi.com/v1
  🔑 API密钥: sk-AGnnT...A14f

✅ 使用统一配置管理成功！


---

## 课程

如果有人叫醒你，然后立即开始问你几个复杂的问题，要求你立即回答，你会表现如何？可能不如给你时间**先思考答案**时表现得好。

猜猜怎么着？大模型也是一样的。

**给大模型时间逐步思考有时会让大模型更准确**，特别是对于复杂任务。

## 🔥 中文提示词逐步思考示例

在中文语境下，逐步思考技术同样非常有效。以下示例展示了如何在中文提示词中应用逐步思考的方法。

### 核心原理
- **让模型慢下来思考**：给大模型时间进行逐步推理
- **结构化思维过程**：通过明确的步骤引导模型思考
- **提高准确性**：特别适用于复杂推理、数学计算、逻辑分析等任务

让我们通过实际示例来学习！


### 示例1：数学问题解决 - 对比基础版与逐步思考版

**背景场景**：解决复杂的数学应用题


In [6]:
# 示例1A：基础版本 - 直接提问
print("📍 基础版本：直接提问数学题")
print("=" * 50)

BASIC_MATH_PROMPT = """
小明的妈妈买了一些苹果。第一天吃掉了总数的1/4，第二天吃掉了剩余的1/3，第三天吃掉了4个，最后还剩下8个苹果。请问小明妈妈一共买了多少个苹果？
"""

response_basic = get_completion(BASIC_MATH_PROMPT)
print("大模型回答：")
print(response_basic)
print("\n" + "="*80 + "\n")


📍 基础版本：直接提问数学题
GPT回答：
### 问题重述

小明的妈妈买了一些苹果。第一天吃掉了总数的1/4，第二天吃掉了剩余的1/3，第三天吃掉了4个，最后还剩下8个苹果。请问小明妈妈一共买了多少个苹果？

### 解题思路

这是一个关于分数和剩余量的问题。为了找到最初购买的苹果总数，我们可以设总数为一个变量，然后根据每天吃掉的数量和剩余的数量，逐步建立方程。

### 设未知数

设小明妈妈一共买了 \( x \) 个苹果。

### 第一天

第一天吃掉了总数的 \( \frac{1}{4} \)，即吃掉了 \( \frac{x}{4} \) 个苹果。

第一天之后剩下的苹果数量为：
\[ x - \frac{x}{4} = \frac{3x}{4} \]

### 第二天

第二天吃掉了剩余的 \( \frac{1}{3} \)，即吃掉了 \( \frac{1}{3} \times \frac{3x}{4} = \frac{x}{4} \) 个苹果。

第二天之后剩下的苹果数量为：
\[ \frac{3x}{4} - \frac{x}{4} = \frac{2x}{4} = \frac{x}{2} \]

### 第三天

第三天吃掉了4个苹果。

第三天之后剩下的苹果数量为：
\[ \frac{x}{2} - 4 \]

### 最后剩余

根据题目，最后还剩下8个苹果，因此：
\[ \frac{x}{2} - 4 = 8 \]

### 解方程

解这个方程：

\[ \frac{x}{2} - 4 = 8 \]
\[ \frac{x}{2} = 8 + 4 \]
\[ \frac{x}{2} = 12 \]
\[ x = 12 \times 2 \]
\[ x = 24 \]

### 验证

让我们验证一下这个答案是否正确。

初始苹果总数：24个。

**第一天：**
吃掉 \( \frac{1}{4} \times 24 = 6 \) 个。
剩余：\( 24 - 6 = 18 \) 个。

**第二天：**
吃掉剩余的 \( \frac{1}{3} \)，即 \( \frac{1}{3} \times 18 = 6 \) 个。
剩余：\( 18 - 6 = 12 \) 个。

**第三天：**
吃掉4个。
剩余：\( 12 - 4 = 8 

In [7]:
# 示例1B：逐步思考版本 - 引导详细推理过程
print("🧠 逐步思考版本：引导详细推理")
print("=" * 50)

STEP_BY_STEP_MATH_PROMPT = """
小明的妈妈买了一些苹果。第一天吃掉了总数的1/4，第二天吃掉了剩余的1/3，第三天吃掉了4个，最后还剩下8个苹果。请问小明妈妈一共买了多少个苹果？

请按照以下步骤一步一步分析：
1. 首先，理解题目条件，设定未知数
2. 然后，分析每天的消费情况
3. 接着，建立数学方程
4. 最后，求解并验证答案

让我一步步来解决这个问题：
"""

response_step_by_step = get_completion(STEP_BY_STEP_MATH_PROMPT)
print("大模型回答：")
print(response_step_by_step)
print("\n" + "="*80 + "\n")


🧠 逐步思考版本：引导详细推理
GPT回答：
### 1. 理解题目条件，设定未知数

首先，我们需要明确题目中给出的信息：

- 妈妈买了一些苹果，设总数为 \( x \) 个。
- 第一天吃掉了总数的 \( \frac{1}{4} \)，即 \( \frac{x}{4} \) 个。
- 第二天吃掉了剩余的 \( \frac{1}{3} \)。
- 第三天吃掉了 4 个。
- 最后还剩下 8 个苹果。

我们的目标是求出 \( x \) 的值。

### 2. 分析每天的消费情况

让我们逐天来看苹果的消耗和剩余情况。

**第一天：**
- 吃掉：\( \frac{x}{4} \)
- 剩余：\( x - \frac{x}{4} = \frac{3x}{4} \)

**第二天：**
- 吃掉剩余的 \( \frac{1}{3} \)，即 \( \frac{1}{3} \times \frac{3x}{4} = \frac{x}{4} \)
- 剩余：\( \frac{3x}{4} - \frac{x}{4} = \frac{2x}{4} = \frac{x}{2} \)

**第三天：**
- 吃掉：4 个
- 剩余：\( \frac{x}{2} - 4 \)

根据题目，最后剩下 8 个苹果，所以：

\[ \frac{x}{2} - 4 = 8 \]

### 3. 建立数学方程

根据上述分析，我们得到方程：

\[ \frac{x}{2} - 4 = 8 \]

### 4. 求解并验证答案

解这个方程：

\[ \frac{x}{2} - 4 = 8 \]
\[ \frac{x}{2} = 8 + 4 \]
\[ \frac{x}{2} = 12 \]
\[ x = 12 \times 2 \]
\[ x = 24 \]

所以，妈妈一共买了 24 个苹果。

**验证：**

让我们验证一下这个答案是否正确。

- 总苹果数：24 个

**第一天：**
- 吃掉：\( \frac{24}{4} = 6 \) 个
- 剩余：\( 24 - 6 = 18 \) 个

**第二天：**
- 吃掉剩余的 \( \frac{1}{3} \)：\( \frac{18}{3} = 6 \) 个
- 剩余：\( 18 - 6 = 12 \) 个

**第三天：

### 示例2：情感分析 - 中文版本的逐步思考

**背景场景**：分析中文评论的情感倾向，特别是含有讽刺或双重含义的内容


In [8]:
# 示例2A：基础版本 - 直接分析情感
print("📍 基础版本：直接分析评论情感")
print("=" * 50)

BASIC_SENTIMENT_PROMPT = """
请分析以下评论的情感倾向（正面/负面/中性）：

"这部电影真是太'精彩'了，我看到一半就睡着了，醒来发现还没结束，真是让人'印象深刻'。演员的表演也很'自然'，自然到我都怀疑他们是不是在念台词。总之，这是一部让我'终生难忘'的作品。"
"""

response_basic_sentiment = get_completion(BASIC_SENTIMENT_PROMPT)
print("大模型回答：")
print(response_basic_sentiment)
print("\n" + "="*60 + "\n")


📍 基础版本：直接分析评论情感
GPT回答：
这段评论的情感倾向是**负面**。尽管评论中使用了“精彩”、“印象深刻”、“自然”、“终生难忘”等看似正面的词汇，但通过上下文和反讽的语气可以看出，作者实际上是在表达对电影的强烈不满和批评。具体表现如下：

1. **反讽手法**：  
   - “精彩”和“印象深刻”实际指电影无聊到让人睡着且冗长。  
   - “自然”的表演被讽刺为僵硬（“怀疑他们是不是在念台词”）。  
   - “终生难忘”并非褒义，而是强调糟糕的观影体验。

2. **直接负面描述**：  
   - “看到一半就睡着了”“醒来发现还没结束”直接表明电影枯燥乏味。  
   - 对演员表演的质疑进一步强化了负面评价。

综上，评论通过反语和直接批评传达了明显的负面情感。




In [9]:
# 示例2B：逐步思考版本 - 结构化分析过程
print("🧠 逐步思考版本：结构化情感分析")
print("=" * 50)

STEP_BY_STEP_SENTIMENT_PROMPT = """
请分析以下评论的情感倾向。请先进行逐步分析，然后给出最终判断。

评论：
"这部电影真是太'精彩'了，我看到一半就睡着了，醒来发现还没结束，真是让人'印象深刻'。演员的表演也很'自然'，自然到我都怀疑他们是不是在念台词。总之，这是一部让我'终生难忘'的作品。"

请按照以下步骤分析：

<步骤1>
首先分析表面词汇：识别评论中使用的形容词和描述性词语
</步骤1>

<步骤2>
然后分析语境线索：注意引号、反讽等修辞手法
</步骤2>

<步骤3>
接着分析具体描述：分析用户提到的具体行为和感受
</步骤3>

<步骤4>
最后综合判断：基于以上分析，判断真实的情感倾向
</步骤4>

让我逐步分析这个评论：
"""

response_step_sentiment = get_completion(STEP_BY_STEP_SENTIMENT_PROMPT)
print("大模型回答：")
print(response_step_sentiment)
print("\n" + "="*60 + "\n")


🧠 逐步思考版本：结构化情感分析
GPT回答：
<步骤1>
首先分析表面词汇：
评论中使用了以下形容词和描述性词语：
- "精彩"
- "印象深刻"
- "自然"
- "终生难忘"
这些词汇表面上看都是积极的、正面的评价。

</步骤1>

<步骤2>
然后分析语境线索：
注意到这些积极的词汇都被加上了引号，这通常表示反讽或讽刺的用法。结合整个句子的语气，可以推测作者是在用这些看似正面的词汇来表达负面的感受。

</步骤2>

<步骤3>
接着分析具体描述：
评论中提到了一些具体的行为和感受：
- "看到一半就睡着了"：表明电影很无聊，无法保持观众的兴趣
- "醒来发现还没结束"：暗示电影过于冗长
- "怀疑他们是不是在念台词"：批评演员表演生硬、不自然
这些具体描述都指向负面的观影体验。

</步骤3>

<步骤4>
最后综合判断：
虽然表面词汇都是积极的，但通过引号的反讽用法和具体的负面体验描述，可以判断作者的真实情感倾向是负面的。作者实际上是在讽刺这部电影无聊、冗长、表演生硬。

最终判断：这个评论的情感倾向是负面的。

</步骤4>




### 示例3：商业决策分析 - 中文商业场景

**背景场景**：帮助企业制定市场策略决策


In [10]:
# 示例3A：基础版本 - 直接提问商业建议
print("📍 基础版本：直接询问商业策略")
print("=" * 50)

BASIC_BUSINESS_PROMPT = """
我们公司是一家生产智能手环的科技公司，目前面临激烈的市场竞争。Apple Watch和华为等大品牌占据了主要市场份额，我们的销量正在下滑。请给出市场策略建议。
"""

response_basic_business = get_completion(BASIC_BUSINESS_PROMPT)
print("GPT回答：")
print(response_basic_business)
print("\n" + "="*60 + "\n")


📍 基础版本：直接询问商业策略
GPT回答：
针对智能手环市场竞争激烈的现状，建议从以下六个维度制定差异化市场策略，逐步突破大品牌封锁：

### 一、精准定位细分市场（避巨头锋芒）
1. **垂直领域深耕**：聚焦大品牌未覆盖的场景（如银发健康监测、儿童安全手环、职业运动员专项训练）
2. **价格错位竞争**：推出199-399元高性价比基础款，同时开发800-1200元专业运动版（参数对标2000元竞品）
3. **区域差异化**：重点突破东南亚/拉美新兴市场，推出防尘防水+超长待机版本

### 二、技术微创新组合拳
1. **独家健康算法**：与三甲医院合作开发房颤/睡眠呼吸暂停预警功能（申请二类医疗器械认证）
2. **场景化续航方案**：基础模式60天续航（行业平均7天）+ 磁吸快充技术
3. **开放式生态**：兼容iOS/Android/鸿蒙三系统，支持微信/支付宝双支付

### 三、渠道下沉+社交裂变
1. **线下体验店**：在三四线城市商场设"健康检测站"（免费体脂测量引流）
2. **KOC计划**：签约1000名健身教练/广场舞队长，佣金翻倍+定制徽章系统
3. **老用户裂变**：旧设备折价换新+分享3人得1年延保

### 四、数据价值变现
1. **企业健康计划**：为中小企业提供员工健康管理SaaS（手环硬件免费+服务收费）
2. **保险合作**：接入平安等保险公司，健康数据达标享保费折扣
3. **科研数据服务**：匿名健康数据出售给医药研究机构

### 五、场景化内容营销
1. **抖音挑战赛**：发起#30天健康挑战#，达标用户抽奖免单
2. **B站测评**：邀请科技UP主做"200元手环 vs 苹果表"极限测试
3. **微信小程序**：开发"家庭健康地图"功能可视化全家数据

### 六、供应链优化
1. **模块化设计**：表带/主机可分离更换，降低二次购买成本
2. **预售+C2M**：京东众筹限量款，销量达标再生产
3. **区域仓布局**：在印度/巴西设KD工厂规避关税

建议优先执行技术微创新+社交裂变组合，6个月内可实现：
- 成本下降20%通过模块化设计
- 用户获取成本降低35%通过KOC计划
- 复购率提升至25%通过健康管理服务

关键数据指标监控：
- 周活跃设备率（反映用户黏性）
- 健

In [11]:
# 示例3B：逐步思考版本 - 结构化商业分析
print("🧠 逐步思考版本：结构化商业策略分析")
print("=" * 50)

STEP_BY_STEP_BUSINESS_PROMPT = """
我们公司是一家生产智能手环的科技公司，目前面临激烈的市场竞争。Apple Watch和华为等大品牌占据了主要市场份额，我们的销量正在下滑。

请按照商业分析框架逐步分析并提出策略建议：

<市场分析>
首先分析当前智能手环市场的竞争格局、用户需求和趋势
</市场分析>

<竞争对手分析>
分析主要竞争对手（Apple Watch、华为等）的优势和劣势
</竞争对手分析>

<自身SWOT分析>
分析我们公司的优势(Strengths)、劣势(Weaknesses)、机会(Opportunities)、威胁(Threats)
</自身SWOT分析>

<差异化策略>
基于分析结果，提出我们的差异化定位和策略
</差异化策略>

<具体执行方案>
给出3-5个具体可执行的市场策略和实施建议
</具体执行方案>

让我按照框架来分析：
"""

response_step_business = get_completion(STEP_BY_STEP_BUSINESS_PROMPT)
print("大模型回答：")
print(response_step_business)
print("\n" + "="*60 + "\n")


🧠 逐步思考版本：结构化商业策略分析
GPT回答：
### <市场分析>
**竞争格局：**  
- 全球智能手环市场呈现"金字塔"结构：高端市场被Apple Watch（市占率超30%）主导，中端市场由华为/小米等把控，低端市场充斥白牌产品。  
- 2023年市场增速放缓至8.7%（IDC数据），但细分领域如医疗级监测、银发族健康管理等仍有20%+增长。  

**用户需求变化：**  
1. **功能需求分层化**：年轻用户关注运动社交功能（如Strava接入），中年群体重视健康预警（ECG/血压），老年用户需要跌倒检测。  
2. **设计时尚化**：42%消费者愿为可更换表带/联名设计支付溢价（Counterpoint调研）。  
3. **续航痛点**：67%用户对Apple Watch的每日充电不满（YouGov数据）。  

**技术趋势：**  
- 无创血糖监测（苹果/三星专利布局中）  
- 卫星紧急通讯（华为北斗卫星消息）  
- 柔性屏技术（小米手环8 Pro已应用）  

### <竞争对手分析>  
| 品牌       | 优势                          | 劣势                          |  
|------------|-------------------------------|-------------------------------|  
| Apple Watch| 生态协同（iPhone联动）、医疗认证（FDA）、品牌溢价 | 续航短、价格高（$399起）、仅适配iOS |  
| 华为       | 精准心率监测、鸿蒙生态、快充技术 | 海外受限（GMS缺失）、高端型号缺eSIM |  
| 小米       | 极致性价比（<￥200）、供应链成本控制 | 数据精度受质疑、毛利仅15%左右 |  

**关键发现**：大品牌在「健康数据权威性」和「生态系统」构建壁垒，但在「垂直场景深耕」和「硬件创新」存在机会窗口。  

### <自身SWOT分析>  
**Strengths**:  
- 快速迭代能力（从设计到量产比大厂快30-60天）  
- 现有医疗合作伙伴（如与三甲医院联合开发心率算法）  

**Weaknesses**:  
- 品牌认知度低（无

### 🎯 逐步思考技巧总结

通过以上中文示例，我们可以看到逐步思考的核心价值：

#### 🔑 关键技巧

1. **使用引导性短语**
   - "让我一步步来解决这个问题："
   - "请按照以下步骤分析："
   - "首先...然后...接着...最后..."

2. **结构化标签**
   - 使用`<步骤1>`、`<分析>`、`<结论>`等XML标签
   - 明确每个步骤的具体任务
   - 保持逻辑递进关系

3. **领域专业框架**
   - 数学：理解→分析→建立方程→求解→验证
   - 商业：市场分析→竞争分析→SWOT→策略→执行
   - 创作：主题→人物→情节→冲突→细节

#### 📈 效果对比

| 方面 | 基础提示 | 逐步思考提示 |
|------|----------|--------------|
| **准确性** | 一般 | 显著提升 |
| **逻辑性** | 跳跃 | 严密连贯 |
| **深度** | 表面 | 深入分析 |
| **可用性** | 需要二次确认 | 直接可用 |

#### 💡 适用场景

- ✅ **复杂推理**：数学计算、逻辑分析
- ✅ **多步骤任务**：商业分析、项目规划
- ✅ **创意工作**：写作、设计、策划
- ✅ **专业分析**：法律、医学、技术评估

---

**💡 记住：给AI时间思考，就像给自己时间思考一样重要！**
